In [25]:
# Code to calculate Niño 3.4 index for CMIP6 data
# Author: Nathan Erickson
# Date: 11/8/2021
# Coded with Python 3.8.10

In [7]:
# Directory management

import os

#dir = '/chinook2/nathane1/Thesis/HadISST/'
dir = '/chinook2/nathane1/Thesis/CMIP6/'
model_list = ["ACCESS-CM2", "ACCESS-ESM1-5", "AWI-CM-1-1-MR", "BCC-CSM2-MR", "CAMS-CSM1-0", "CanESM5", "CESM2", "CESM2-WACCM", "CMCC-CM2-SR5", 
              "CNRM-CM6-1", "CNRM-CM6-1-HR", "CNRM-ESM2-1", "EC-Earth3", "EC-Earth3-Veg", "FGOALS-f3-L", "FGOALS-g3", "GFDL-CM4", "GFDL-ESM4", 
              "GISS-E2-1-G", "HadGEM3-GC31-LL", "HadGEM3-GC31-MM", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", "MIROC6", "MIROC-ES2L",
              "MPI-ESM1-2-HR", "MPI-ESM1-2-LR", "MPI-ESM2-0", "NESM3", "NorESM2-LM", "NorESM2-MM", "TaiESM1"]
model = 'ACCESS-CM2'
path = dir + model
os.chdir(path)

#Set up time

import time
start_time = time.time()

In [8]:
import numpy as np
import pandas as pd
import xarray as xr
import cftime as cf
import warnings

warnings.simplefilter("ignore","SerializationWarning:")

In [17]:
# Read in files from working directory
# Read in model data

data = xr.open_dataset(f'ts_Amon_{model}_histssp585_r1i1p1f1_gn_185001-210012.nc')
#data = xr.open_dataset('/home/nathane1/Thesis/sst.mnmean.nc')

# Read in land mask

land_mask = xr.open_dataset(f'sftlf_fx_{model}_historical_r1i1p1f1_gn.nc')
data

<xarray.Dataset>
Dimensions:    (time: 3012, bnds: 2, lat: 144, lon: 192)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2100-12-16T12:00:00
  * lat        (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * lon        (lon) float64 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    ts         (time, lat, lon) float32 ...
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    creation_date:          2019-11-08T06:41:46Z
    ...                     ...
    variant_label:          r1i1p1f1
    version:                v20191108
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/246cb7d2-3eb7-4127-a23f-35467313072f
    license:                CMIP6 model data produced by CSIRO is licensed un...
    NCO:                    netCDF Operators version 4.7.9 (Homepage = http:/...

In [18]:
# Assign data arrays to variables; mask out land surfaces

lon = data['lon']
lat = data['lat']
ts = data['ts']

land_masked = ts.where(land_mask['sftlf'] != 100)

In [19]:
average_ts = land_masked.mean(['lat','lon'])
avg_timesteps = 31
#average_ts = ts.mean(['lat','lon'])

average_jan = average_ts.loc[[np.datetime_as_string(average_ts['time'].values)[i][5:7]  == '01' for i in range(len(average_ts['time'].values))]]
average_feb = average_ts.loc[[np.datetime_as_string(average_ts['time'].values)[i][5:7]  == '02' for i in range(len(average_ts['time'].values))]]
average_dec = average_ts.loc[[np.datetime_as_string(average_ts['time'].values)[i][5:7]  == '12' for i in range(len(average_ts['time'].values))]]

clima_jan = average_jan.rolling(time = avg_timesteps, center = True).mean()
clima_feb = average_feb.rolling(time = avg_timesteps, center = True).mean()
clima_dec = average_dec.rolling(time = avg_timesteps, center = True).mean()

clima_jan[0:15] = clima_jan[0:15].fillna(value = clima_jan[16])
clima_jan[-15:] = clima_jan[-15:].fillna(value = clima_jan[-16])
clima_feb[0:15] = clima_feb[0:15].fillna(value = clima_feb[16])
clima_feb[-15:] = clima_feb[-15:].fillna(value = clima_feb[-16])
clima_dec[0:15] = clima_dec[0:15].fillna(value = clima_dec[16])
clima_dec[-15:] = clima_dec[-15:].fillna(value = clima_dec[-16])

In [106]:
jan_anom = float((average_jan[0] - clima_jan[0]).values)
feb_anom = float((average_feb[0] - clima_feb[0]).values)
dec_anom = float((average_dec[0] - clima_dec[0]).values)

jan_anom

-0.16693312121975623

In [27]:
def calculate_niño(ssts): 
    sst_anomalies= []
    
    # Select Niño 3.4 region
    ts_nino = ssts.sel(lon=slice(120,170),lat=slice(-5,5)) # <- Make sure to change me when switching between obs/model runs
    average_ts = ts_nino.mean(['lat','lon'])
    
    # Establish a background climatology to use for Niño 3.4 calculation
    avg_timesteps = 5 # Need to generalize the amount of timesteps
    average_jan = average_ts.loc[[np.datetime_as_string(average_ts['time'].values)[i][5:7]  == '01' for i in range(len(average_ts['time'].values))]]
    average_feb = average_ts.loc[[np.datetime_as_string(average_ts['time'].values)[i][5:7]  == '02' for i in range(len(average_ts['time'].values))]]
    average_dec = average_ts.loc[[np.datetime_as_string(average_ts['time'].values)[i][5:7]  == '12' for i in range(len(average_ts['time'].values))]]

    clima_jan = average_jan.rolling(time = avg_timesteps, center = True).mean()
    clima_feb = average_feb.rolling(time = avg_timesteps, center = True).mean()
    clima_dec = average_dec.rolling(time = avg_timesteps, center = True).mean()

    clima_jan[0:15] = clima_jan[0:15].fillna(value = clima_jan[16])
    clima_jan[-15:] = clima_jan[-15:].fillna(value = clima_jan[-16])
    clima_feb[0:15] = clima_feb[0:15].fillna(value = clima_feb[16])
    clima_feb[-15:] = clima_feb[-15:].fillna(value = clima_feb[-16])
    clima_dec[0:15] = clima_dec[0:15].fillna(value = clima_dec[16])
    clima_dec[-15:] = clima_dec[-15:].fillna(value = clima_dec[-16])
    
    # Calculate Niño 3.4 index for the dataset
    for t in range(0,len(ts_nino)):
        try:
            jan_anom = float((average_jan[t] - clima_jan[t]).values)
            feb_anom = float((average_feb[t] - clima_feb[t]).values)
            dec_anom = float((average_dec[t] - clima_dec[t]).values)
            sst_anomalies.append(jan_anom)
            sst_anomalies.append(feb_anom)
            sst_anomalies.append(dec_anom)
        except:
            pass
    
    return sst_anomalies

In [ ]:
sst_anomalies = calculate_niño(land_masked)
sst_anomalies

In [29]:
pd.Series(sst_anomalies, name = 'ACCESS-CM2-r1p1i1f1')#.to_csv('/home/nathane1/Thesis/output/ERSST_v5.csv')

0      0.104810
1      0.174115
2      0.025543
3      0.075848
4      0.182294
         ...   
748    0.790472
749    0.816345
750    0.827118
751    0.763892
752    1.100769
Name: ACCESS-CM2-r1p1i1f1, Length: 753, dtype: float64

In [30]:
nino_output = pd.DataFrame()
nino_output.insert(0,f'{model}_histssp585_r1i1p1f1',sst_anomalies)
os.chdir('/home/nathane1/Thesis/')
nino_output.to_csv('output/niño_3.4_table.csv')